In [1]:
import itertools as it
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy.misc
import scipy.optimize as op
import scipy.io
import random
from IPython.display import display, Math, Latex

# set path
import os
os.chdir('%s/courses/coursera_bigdata/course3/week2' % os.getenv('DST'))
print(os.getcwd())

# printing options 
np.set_printoptions(linewidth=250)
np.set_printoptions(precision=3)

%matplotlib inline
# plotting options
plt.rc('font'  , size=18)
plt.rc('figure', figsize=(10, 8))
plt.rc('axes'  , labelsize=22)
plt.rc('legend', fontsize=16)
plt.rc('figure', figsize=(10, 8))

/Users/rwk7t/Development/dst/courses/coursera_bigdata/course3/week2


# Import data

In [2]:
def lower_column_names(df):
    df.columns = [c.replace(' ', '_').lower() for c in df.columns]
    return 

In [3]:
df1 = pd.read_csv('bike_data/201402_trip_data.csv'); lower_column_names(df1)
df2 = pd.read_csv('bike_data/201408_trip_data.csv'); lower_column_names(df2)
df3 = pd.read_csv('bike_data/201508_trip_data.csv'); lower_column_names(df3)

In [4]:
min(df1.end_date), min(df2.end_date), min(df3.end_date)

('1/1/2014 0:21', '3/1/2014 0:17', '1/1/2015 0:30')

In [5]:
max(df1.end_date), max(df2.end_date), max(df3.end_date)

('9/9/2013 9:57', '9/1/2014 8:08', '9/9/2014 9:59')

In [6]:
df = pd.concat([df1, df2, df3])
df.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
0,520,63,8/29/2013 14:14,South Van Ness at Market,66,8/29/2013 14:13,South Van Ness at Market,66,NaN,Subscriber,4576,94127
1,661,70,8/29/2013 14:43,San Jose City Hall,10,8/29/2013 14:42,San Jose City Hall,10,NaN,Subscriber,4607,95138
2,48,71,8/29/2013 10:17,Mountain View City Hall,27,8/29/2013 10:16,Mountain View City Hall,27,NaN,Subscriber,4130,97214


In [7]:
# clean up

del(df1)
del(df2)
del(df3)

# quesiton 1:

Utilizing the Bay Area Bike Share database - what is the most popular start station based on trip data?


In [8]:
g = df.groupby('start_station', sort=True).count().sort('duration', ascending=False)
g.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
start_station,,,,,,,,,,,
San Francisco Caltrain (Townsend at 4th),49092,49092,49092,49092,49092,49092,49092,39254,9838,49092,48916
San Francisco Caltrain 2 (330 Townsend),33742,33742,33742,33742,33742,33742,33742,28766,4976,33742,33658
Harry Bridges Plaza (Ferry Building),32934,32934,32934,32934,32934,32934,32934,25591,7343,32934,32312


In [9]:
g.head(1).index.values[0]

'San Francisco Caltrain (Townsend at 4th)'

# Question 2:

Which start station has the longest trip duration?

In [10]:
q2 = df.sort('duration', ascending=False).head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
257759,535,17270400,6/24/2015 20:18,2nd at Folsom,62,12/6/2014 21:59,South Van Ness at Market,66,Customer,NaN,568474,95531
66911,466,2137000,7/23/2015 15:27,Yerba Buena Center of the Arts (3rd @ Howard),68,6/28/2015 21:50,Market at Sansome,77,Customer,NaN,825850,97213
124532,680,1852590,5/23/2015 16:53,Castro Street and El Camino Real,32,5/2/2015 6:17,San Antonio Shopping Center,31,Subscriber,NaN,750192,94024


# Question 3:

Utilizing the Bay Area Bike Share database Year 1 - what is the Second most popular end station based on trip data ?

In [11]:
# just load year 1 data
df1 = pd.read_csv('bike_data/201402_trip_data.csv'); lower_column_names(df1)
df2 = pd.read_csv('bike_data/201408_trip_data.csv'); lower_column_names(df2)
df = pd.concat([df1, df2])
df.head(3)

,bike_#,duration,end_date,end_station,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
0,520,63,8/29/2013 14:14,South Van Ness at Market,66,8/29/2013 14:13,South Van Ness at Market,66,NaN,Subscriber,4576,94127
1,661,70,8/29/2013 14:43,San Jose City Hall,10,8/29/2013 14:42,San Jose City Hall,10,NaN,Subscriber,4607,95138
2,48,71,8/29/2013 10:17,Mountain View City Hall,27,8/29/2013 10:16,Mountain View City Hall,27,NaN,Subscriber,4130,97214


In [12]:
del(df1)
del(df2)

In [13]:
g = df.groupby('end_station', sort=True).count().sort('duration', ascending=False)
g.head(3)

,bike_#,duration,end_date,end_terminal,start_date,start_station,start_terminal,subscriber_type,subscription_type,trip_id,zip_code
end_station,,,,,,,,,,,
San Francisco Caltrain (Townsend at 4th),28369,28369,28369,28369,28369,28369,28369,16732,11637,28369,28190
Embarcadero at Sansome,15731,15731,15731,15731,15731,15731,15731,8141,7590,15731,14764
Harry Bridges Plaza (Ferry Building),15383,15383,15383,15383,15383,15383,15383,7908,7475,15383,14882
